In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from rdkit.Chem import PandasTools
from rdkit import rdBase
from rdkit import Chem
from rdkit.Chem.rdmolfiles import SmilesWriter
from pathlib import Path
from rdkit.Chem import Descriptors, Draw, PandasTools
from warnings import filterwarnings
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from FP import *

In [2]:
# Set path to this notebook
HERE = Path(_dh[-1])
HDAC6 = Path(HERE).resolve().parents[0]/ "BioactivityModels/HDAC6/input"
HDAC1 = Path(HERE).resolve().parents[0] / "BioactivityModels/HDAC1/input"
HDAC6vs1 = Path(HERE).resolve().parents[0] / "SelectivityModels/input"

#### Generate the fingerprints for the training/test set as well as external test set.
Use different types of FPs and develop models with each one of them to test 

#### for HDAC6 dataset

In [13]:
df6All = pd.read_csv(HDAC6/'HDAC6_dataset.csv', )
#1024 bits fingerprints and MACCS for all compounds
compound_df6All = df6All.copy()
compound_df6All["fp_MACCS"] = compound_df6All['smiles'].apply(smiles_to_maccs)
compound_df6All["fp_Morgan3"] = compound_df6All['smiles'].apply(smiles_to_morgan3)
compound_df6All["fp_MorganF"] = compound_df6All['smiles'].apply(smiles_to_morganF)
compound_df6All["fp_MAP4"] = compound_df6All['smiles'].apply(smiles_to_MAP4)
compound_df6All = compound_df6All[['molecule_chembl_id', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','pChEMBL_HDAC6' ]]
compound_df6All.to_pickle(HDAC6/"HDAC6_1024B.csv")
compound_df6All.head(5)

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,pChEMBL_HDAC6
0,CHEMBL4082520,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4332149, 1499838, 4703614, 2817337, 4024710, ...",10.10
1,CHEMBL4098975,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, ...","[2770663, 83582, 137380, 5467685, 28861, 20399...",9.85
2,CHEMBL4100534,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4681989, 1499838, 559823, 2817337, 4024710, 6...",9.82
3,CHEMBL4101480,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4681989, 2349897, 559823, 2817337, 4024710, 6...",9.80
4,CHEMBL3912061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 4142580, 576166, 5215404, 6...",9.77


In [4]:
#2048 bits fingerprints for all compounds
compound_df6All2 = df6All.copy()
compound_df6All2["fp_Morgan3"] = compound_df6All2['smiles'].apply(smiles_to_morgan3_2048)
compound_df6All2["fp_MorganF"] = compound_df6All2['smiles'].apply(smiles_to_morganF_2048)
compound_df6All2["fp_MAP4"] = compound_df6All2['smiles'].apply(smiles_to_MAP4_2048)
compound_df6All2 = compound_df6All2[['molecule_chembl_id', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','pChEMBL_HDAC6' ]]
compound_df6All2.to_pickle(HDAC6/"HDAC6_2048B.csv")
compound_df6All2.head(5)

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,pChEMBL_HDAC6
0,CHEMBL4082520,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4332149, 1499838, 4703614, 2817337, 4024710, ...",10.10
1,CHEMBL4098975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2770663, 83582, 137380, 5467685, 28861, 20399...",9.85
2,CHEMBL4100534,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4681989, 1499838, 559823, 2817337, 4024710, 6...",9.82
3,CHEMBL4101480,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4681989, 2349897, 559823, 2817337, 4024710, 6...",9.80
4,CHEMBL3912061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 4142580, 576166, 5215404, 6...",9.77


#### for HDAC1 dataset

In [6]:
df1All = pd.read_csv(HDAC1/'HDAC1_dataset.csv', )
#1024 bits fingerprints and MACCS
compound_df1All = df1All.copy()
compound_df1All["fp_MACCS"] = compound_df1All['smiles'].apply(smiles_to_maccs)
compound_df1All["fp_Morgan3"] = compound_df1All['smiles'].apply(smiles_to_morgan3)
compound_df1All["fp_MorganF"] = compound_df1All['smiles'].apply(smiles_to_morganF)
compound_df1All["fp_MAP4"] = compound_df1All['smiles'].apply(smiles_to_MAP4)
compound_df1All = compound_df1All[['molecule_chembl_id','fp_MACCS', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','pChEMBL_HDAC1' ]]
compound_df1All.to_pickle(HDAC1/"HDAC1_1024B.csv")
compound_df1All.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,pChEMBL_HDAC1
0,CHEMBL327146,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[5547132, 3346815, 526260, 7125875, 7332227, 1...",9.00
1,CHEMBL116620,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[2593493, 2349897, 526260, 13440577, 19258323,...",9.00
2,CHEMBL2093007,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5090223, 4497230, 396665, 4659087, 418066, 30...",5.20
3,CHEMBL316457,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[9076848, 10234695, 1615018, 4730798, 2140073,...",6.22
4,CHEMBL269692,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 29775619, 8033062, 11956864, 7698299...",5.52


In [7]:
#2048 bits fingerprints for all dataset
compound_df1All2 = df1All.copy()
compound_df1All2["fp_Morgan3"] = compound_df1All2['smiles'].apply(smiles_to_morgan3_2048)
compound_df1All2["fp_MorganF"] = compound_df1All2['smiles'].apply(smiles_to_morganF_2048)
compound_df1All2["fp_MAP4"] = compound_df1All2['smiles'].apply(smiles_to_MAP4_2048)
compound_df1All2 = compound_df1All2[['molecule_chembl_id', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','pChEMBL_HDAC1' ]]
compound_df1All2.to_pickle(HDAC1/"HDAC1_2048B.csv")
compound_df1All2.head(5)

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,pChEMBL_HDAC1
0,CHEMBL327146,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[5547132, 3346815, 526260, 7125875, 7332227, 1...",9.00
1,CHEMBL116620,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2593493, 2349897, 526260, 13440577, 19258323,...",9.00
2,CHEMBL2093007,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5090223, 4497230, 396665, 4659087, 418066, 30...",5.20
3,CHEMBL316457,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[9076848, 10234695, 1615018, 4730798, 2140073,...",6.22
4,CHEMBL269692,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 29775619, 8033062, 11956864, 7698299...",5.52


#### for HDAC1/6 dataset

In [8]:
df5 = pd.read_csv(HDAC6vs1/'HDAC1and6_SemiSel_dataset.csv', )
#1024 bits fingerprints and MACCS
compound_df9 = df5.copy()
compound_df9["fp_MACCS"] = compound_df9['smiles'].apply(smiles_to_maccs)
compound_df9["fp_Morgan3"] = compound_df9['smiles'].apply(smiles_to_morgan3)
compound_df9["fp_MorganF"] = compound_df9['smiles'].apply(smiles_to_morganF)
compound_df9["fp_MAP4"] = compound_df9['smiles'].apply(smiles_to_MAP4)
compound_df9 = compound_df9[['molecule_chembl_id','fp_MACCS', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','SelectivityWindow']]
compound_df9.to_pickle(HDAC6vs1/"HDAC1and6_SemiSel_1024B.csv")
compound_df9.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL4098975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, ...","[2770663, 83582, 137380, 5467685, 28861, 20399...",2.89
1,CHEMBL3912061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 4142580, 576166, 5215404, 6...",3.56
2,CHEMBL4243347,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[6411541, 3821889, 137380, 4236434, 7332227, 2...",1.00
3,CHEMBL4247128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[3808044, 3821889, 137380, 2385390, 6940687, 2...",2.52
4,CHEMBL4126811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[4928228, 22758228, 8033062, 10725227, 1087409...",3.12


In [9]:
#2048 bits fingerprints
compound_df10 = df5.copy()
compound_df10["fp_Morgan3"] = compound_df10['smiles'].apply(smiles_to_morgan3_2048)
compound_df10["fp_MorganF"] = compound_df10['smiles'].apply(smiles_to_morganF_2048)
compound_df10["fp_MAP4"] = compound_df10['smiles'].apply(smiles_to_MAP4_2048)
compound_df10 = compound_df10[['molecule_chembl_id', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','SelectivityWindow' ]]
compound_df10.rename({'sel6':'selectivityIndex'}, axis=1, inplace=True)
compound_df10.to_pickle((HDAC6vs1/"HDAC1and6_SemiSel_2048B.csv"))
compound_df10.head(5)

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL4098975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2770663, 83582, 137380, 5467685, 28861, 20399...",2.89
1,CHEMBL3912061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 4142580, 576166, 5215404, 6...",3.56
2,CHEMBL4243347,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[6411541, 3821889, 137380, 4236434, 7332227, 2...",1.00
3,CHEMBL4247128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[3808044, 3821889, 137380, 2385390, 6940687, 2...",2.52
4,CHEMBL4126811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4928228, 22758228, 8033062, 10725227, 1087409...",3.12


#### HDAC1/6 set without Semi-selective compounds

In [10]:
df7 = pd.read_csv(HDAC6vs1/'HDAC1and6_dataset.csv' )
#1024 bits fingerprints and MACCS
compound_df12 = df7.copy()
compound_df12["fp_MACCS"] = compound_df12['smiles'].apply(smiles_to_maccs)
compound_df12["fp_Morgan3"] = compound_df12['smiles'].apply(smiles_to_morgan3)
compound_df12["fp_MorganF"] = compound_df12['smiles'].apply(smiles_to_morganF)
compound_df12["fp_MAP4"] = compound_df12['smiles'].apply(smiles_to_MAP4)
compound_df12 = compound_df12[['molecule_chembl_id','fp_MACCS', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','SelectivityWindow' ]]
compound_df12.to_pickle(HDAC6vs1/"HDAC1and6_1024B.csv")
compound_df12.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL4098975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, ...","[2770663, 83582, 137380, 5467685, 28861, 20399...",2.89
1,CHEMBL3912061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 4142580, 576166, 5215404, 6...",3.56
2,CHEMBL4243347,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[6411541, 3821889, 137380, 4236434, 7332227, 2...",1.00
3,CHEMBL4247128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[3808044, 3821889, 137380, 2385390, 6940687, 2...",2.52
4,CHEMBL4126811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[4928228, 22758228, 8033062, 10725227, 1087409...",3.12


In [11]:
#2048 bits fingerprints
compound_df13 = df7.copy()
compound_df13["fp_Morgan3"] = compound_df13['smiles'].apply(smiles_to_morgan3_2048)
compound_df13["fp_MorganF"] = compound_df13['smiles'].apply(smiles_to_morganF_2048)
compound_df13["fp_MAP4"] = compound_df13['smiles'].apply(smiles_to_MAP4_2048)
compound_df13 = compound_df13[['molecule_chembl_id', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','SelectivityWindow' ]]
compound_df13.to_pickle((HDAC6vs1/"HDAC1and6_2048B.csv"))
compound_df13.head(5)

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL4098975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[2770663, 83582, 137380, 5467685, 28861, 20399...",2.89
1,CHEMBL3912061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5976924, 3913535, 4142580, 576166, 5215404, 6...",3.56
2,CHEMBL4243347,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[6411541, 3821889, 137380, 4236434, 7332227, 2...",1.00
3,CHEMBL4247128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[3808044, 3821889, 137380, 2385390, 6940687, 2...",2.52
4,CHEMBL4126811,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4928228, 22758228, 8033062, 10725227, 1087409...",3.12


#### External set molecules

In [14]:
dfext = pd.read_csv(HDAC6vs1/'HDAC1and6_external_420Mols.csv',)
#1024 bits fingerprints and MACCS for all compounds
external = dfext.copy()
external.rename({'sel6':'SelectivityWindow'}, axis=1, inplace=True)
external["fp_MACCS"] = external['smiles'].apply(smiles_to_maccs)
external["fp_Morgan3"] = external['smiles'].apply(smiles_to_morgan3)
external["fp_MorganF"] = external['smiles'].apply(smiles_to_morganF)
external["fp_MAP4"] = external['smiles'].apply(smiles_to_MAP4)
external = external[['molecule_chembl_id','fp_MACCS', 'fp_Morgan3', 'fp_MorganF', 'fp_MAP4','SelectivityWindow' ]]
external.to_pickle(HDAC6vs1/"HDAC1and6_external_420Mols_1024B.csv")
external.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL4442817,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[727366, 2349897, 137380, 18248611, 6944096, 4...",0.7754
1,CHEMBL4455447,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[6411541, 2349897, 137380, 18248611, 7332227, ...",0.2951
2,CHEMBL4463358,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[727366, 2349897, 137380, 18248611, 135217, 29...",0.7392
3,CHEMBL4463948,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[6411541, 2349897, 137380, 14079558, 7332227, ...",0.7707
4,CHEMBL4465542,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[727366, 2349897, 137380, 12231903, 7332227, 4...",0.7379
